In [1]:
!pip install imutils

In [2]:
import tensorflow as tf 
import cv2 
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.python.keras.utils import generic_utils
from tensorflow.keras import models
from tensorflow.keras import layers

In [3]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Aug  3 12:24:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive  #This is to make a short form for the mydrive location 
!ls /mydrive

Mounted at /content/gdrive
'200001044 (1).pdf'
'200001044 (2).pdf'
 200001044_A1_PH156_ExpNo8.pdf
 200001044_A1_PH156_Exp.No.9.pdf
 200001044.jpg
'200001044_Mihir Karandikar_PH156_Exp3.pdf'
'200001044_Mihir Karandikar_PH156_ExpNo2.pdf'
 200001044.pdf
 200001044_PH156_Exp4_FourProbe.pdf
 200001044_PH156_Exp5_KundtsTube.pdf
'200001044_Vaccine Certificate.pdf'
 Classroom
'CodeChef Problem.cpp'
'Colab Notebooks'
'cse200001044_MIHIR KANCHAN KARANDIKAR.pdf'
'EG Lab(1) (01-12-2020) (200001044)  (Mihir Karandikar).pdf'
'EG lab 2_A2_2021.pdf'
 Emotion_Detection
 IITISoc21_GroupFolder
"IITISoc'21 Mask Detection.gslides"
 IITISoc_Images_270621.zip
'IITISoc Proposal.pdf'
 output.pdf
 program.pdf
 VID_20201220_145423.mp4
 YOLO_IITISOC


In [5]:
# !unzip /mydrive/Emotion_Detection/Training_Dataset.zip -d /mydrive/Emotion_Detection/Training_Images 

Streaming output truncated to the last 5000 lines.
 extracting: /mydrive/Emotion_Detection/Training_Images/Sad/PrivateTest_8089222.jpg  
 extracting: /mydrive/Emotion_Detection/Training_Images/Sad/PrivateTest_831970.jpg  
 extracting: /mydrive/Emotion_Detection/Training_Images/Sad/PrivateTest_8421425.jpg  
 extracting: /mydrive/Emotion_Detection/Training_Images/Sad/PrivateTest_8808891.jpg  
 extracting: /mydrive/Emotion_Detection/Training_Images/Sad/PrivateTest_8886048.jpg  
 extracting: /mydrive/Emotion_Detection/Training_Images/Sad/PrivateTest_9382351.jpg  
 extracting: /mydrive/Emotion_Detection/Training_Images/Sad/PrivateTest_9856991.jpg  
 extracting: /mydrive/Emotion_Detection/Training_Images/Sad/PrivateTest_9859012.jpg  
 extracting: /mydrive/Emotion_Detection/Training_Images/Sad/PrivateTest_9966612.jpg  
 extracting: /mydrive/Emotion_Detection/Training_Images/Sad/PublicTest_76990999.jpg  
 extracting: /mydrive/Emotion_Detection/Training_Images/Sad/PublicTest_76999798.jpg  
 ext

In [6]:
# img_size=224
# sample_img=cv2.imread("")
# print(sample_img.shape)
# plt.imshow(sample_img)
# resized_sample=cv2.resize(sample_img,(img_size,img_size))
# plt.imshow(resized_sampl)

In [7]:
DataDirectory="/mydrive/Emotion_Detection/Training_Images"
Classes=["Happy","Sad","Scared","Surprised"]

training_Data= []

img_size=224

def Create_Training_Data():
    for category in Classes:
        path=os.path.join(DataDirectory,category)
        class_num=Classes.index(category)  #0,1,2....Labels
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(img_size,img_size))
                training_Data.append([new_array,class_num])
            except Exception as e:
                pass


Create_Training_Data()

import random
random.shuffle(training_Data)

X=[] #A list
y=[] #A list

for features,label in training_Data:
    X.append(features)
    y.append(label)

X=np.array(X).reshape(-1,img_size,img_size,3)  #Design Matrix size (shape= #no.of.images x pixels x pixels x 3)  
                                               #We put-1 so as to automatically detect the number of images

#Normalize the data
X=X/255.0 #Other way to normalize=Using scikit learn

Y=np.array(y) #Label Matrix (shape= #no.of.images x 1 )



In [12]:
print(X.shape)
print(Y.shape)
print(y[1])
print(y[2])
print(y[3])
print(y[4])
print(len(training_Data))
temp=np.array(training_Data)
temp.shape #This should be (no.of.images,2)

(7418, 224, 224, 3)
(7418,)
3
0
0
0
7418


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


(7418, 2)

In [13]:
model=tf.keras.applications.MobileNetV2() #Pretriained model
model.summary()

14540800/14536120 [==============================] - 1s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________

In [14]:
#Remove/Modify some things in the huge model
base_input=model.layers[0].input   #'0' means that for input we are taking the first layer of the model
base_output=model.layers[-2].output #Discard the last layer i.e. -1 from the Pretrained model...start from -2

final_output=layers.Dense(128)(base_output)
final_output=layers.Activation('relu')(final_output)
final_output=layers.Dense(64)(final_output)
final_output=layers.Activation('relu')(final_output)
final_output= layers.Dense(4,activation='softmax')(final_output)  #3 classes

In [15]:
new_model=keras.Model(inputs=base_input,outputs=final_output)
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [16]:
new_model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])  #Settings for multi-class classification

In [ ]:
new_model.fit(X,Y,epochs=15)

In [ ]:
new_model.save('/mydrive/Emotion_Detection/emotion_detector.h5')

In [ ]:
sample_test_img=cv2.imread("/mydrive/Emotion_Detection/")
plt.imshow(cv2.cvtColor(sample_test_img,cv2.COLOR_BGR2RGB)) #To display a single image just as an example
plt.show()

final_sample_test_img=cv2.resize(sample_test_img,(224,224))
final_sample_test_img=np.expand_dims(final_sample_test_img,axis=0) #We need a fourth dimension #This is similar to using -1
final_sample_test_img=final_sample_test_img/255.0

In [ ]:
Predictions=new_model.predict(final_sample_test_img)
Predictions

In [ ]:
#Testing the Model   trained_model.evaluate #For getting metrics on test data set